In [1]:
import numpy as np
import argparse
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [2]:
# command line output
ap = argparse.ArgumentParser()
ap.add_argument("-f","--mode",help="train/display")  #this -f is to a way to fool the argument
mode = ap.parse_args().mode

In [17]:
# plots accuracy and loss curves
import matplotlib.pyplot as plt
def plot_model_history(model_history):
    """
    Plot Accuracy and Loss curves given the model_history
    """
    fig, axs = plt.subplots(1,2,figsize=(15,5))
    # summarize history for accuracy
    axs[0].plot(range(1,len(model_history.history['accuracy'])+1),model_history.history['accuracy'])
    axs[0].plot(range(1,len(model_history.history['val_accuracy'])+1),model_history.history['val_accuracy'])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(model_history.history['accuracy'])+1),len(model_history.history['accuracy'])/10)
    axs[0].legend(['train', 'val'], loc='best')
    # summarize history for loss
    axs[1].plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    axs[1].plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1),len(model_history.history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')
    fig.savefig('plot.png')
    plt.show()

In [4]:
# Define data generators
train_dir = 'data/train'
val_dir = 'data/test'

In [5]:
num_train = 28709
num_val = 7178
batch_size = 64
num_epoch = 50

In [6]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

In [7]:
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(48,48),
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(48,48),
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical')

Found 86911 images belonging to 7 classes.
Found 7184 images belonging to 7 classes.


In [8]:
# Create the model
model = Sequential()
print (model)

In [9]:
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
print (model)

In [10]:
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
print (model)

In [11]:
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))
print (model)

In [13]:
mode="display"

In [14]:
# If you want to train the same model or try other models, go for this
if mode == "train":
    model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0001, decay=1e-6),metrics=['accuracy'])
    model_info = model.fit_generator(
            train_generator,
            steps_per_epoch=num_train // batch_size,
            epochs=num_epoch,
            validation_data=validation_generator,
            validation_steps=num_val // batch_size)
     plot_model_history(model_info)
    model.save_weights('model.h5')
    
# emotions will be displayed on your face from the webcam feed
elif mode == "display":
    model.load_weights('model.h5')

    # prevents openCL usage and unnecessary logging messages
    cv2.ocl.setUseOpenCL(False)

    # dictionary which assigns each label an emotion (alphabetical order)
    emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

    # start the webcam feed
    cap = cv2.VideoCapture(0)
    while True:
        # Find haar cascade to draw bounding box around face
        ret, frame = cap.read()
        if not ret:
            break
        facecasc = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = facecasc.detectMultiScale(gray,scaleFactor=1.3, minNeighbors=5)

        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
            roi_gray = gray[y:y + h, x:x + w]
            cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
            prediction = model.predict(cropped_img)
            maxindex = int(np.argmax(prediction))
            cv2.putText(frame, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        cv2.imshow('Video', cv2.resize(frame,(1600,960),interpolation = cv2.INTER_CUBIC))
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [14]:
model.save_weights('model.h5')

In [18]:
plot_model_history(model_info)

NameError: name 'model_info' is not defined

In [15]:
mode="display"

In [21]:
# If you want to train the same model or try other models, go for this
if mode == "train":
    model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0001, decay=1e-6),metrics=['accuracy'])
    model_info = model.fit_generator(
            train_generator,
            steps_per_epoch=num_train // batch_size,
            epochs=num_epoch,
            validation_data=validation_generator,
            validation_steps=num_val // batch_size)
    plot_model_history(model_info)
    model.save_weights('model.h5')
    
# emotions will be displayed on your face from the webcam feed
elif mode == "display":
    model.load_weights('model.h5')

    # prevents openCL usage and unnecessary logging messages
    cv2.ocl.setUseOpenCL(False)

    # dictionary which assigns each label an emotion (alphabetical order)
    emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

    # start the webcam feed
    cap = cv2.VideoCapture(0)
    while True:
        # Find haar cascade to draw bounding box around face
        ret, frame = cap.read()
        if not ret:
            break
        print (frame)
        facecasc = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        print (gray)
        faces = facecasc.detectMultiScale(gray,scaleFactor=1.3, minNeighbors=5)

        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
            roi_gray = gray[y:y + h, x:x + w]
            cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
            prediction = model.predict(cropped_img)
            maxindex = int(np.argmax(prediction))
            cv2.putText(frame, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        cv2.imshow('Video', cv2.resize(frame,(1600,960),interpolation = cv2.INTER_CUBIC))
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    

[[[128  85 139]
  [128  85 139]
  [127  86 141]
  ...
  [255 200 255]
  [255 199 255]
  [255 201 255]]

 [[129  85 142]
  [129  85 142]
  [127  85 142]
  ...
  [255 200 255]
  [255 198 255]
  [255 198 255]]

 [[134  86 146]
  [131  84 144]
  [130  87 146]
  ...
  [255 199 255]
  [255 197 255]
  [255 197 255]]

 ...

 [[ 93  67 110]
  [ 94  68 111]
  [ 97  67 115]
  ...
  [ 81  52 112]
  [ 76  53 107]
  [ 76  53 107]]

 [[ 90  64 109]
  [ 90  64 109]
  [ 99  64 113]
  ...
  [ 80  53 107]
  [ 75  53 105]
  [ 76  54 106]]

 [[ 94  63 109]
  [ 94  63 109]
  [103  63 113]
  ...
  [ 78  51 103]
  [ 73  52  99]
  [ 75  55 101]]]
[[106 106 107 ... 223 222 223]
 [107 107 107 ... 223 222 222]
 [109 107 110 ... 222 221 221]
 ...
 [ 83  84  85 ...  73  72  72]
 [ 80  80  83 ...  72  71  72]
 [ 80  80  83 ...  70  68  71]]
[[[117  90 136]
  [120  93 138]
  [113  90 139]
  ...
  [255 188 255]
  [255 191 255]
  [255 195 255]]

 [[117  90 136]
  [117  90 136]
  [115  89 139]
  ...
  [255 189 255]
  [2

[[ 99 100 100 ... 211 212 215]
 [ 99  99  99 ... 209 209 212]
 [ 99  99  99 ... 208 208 209]
 ...
 [ 76  73  72 ...  65  63  63]
 [ 75  73  73 ...  64  65  63]
 [ 78  73  75 ...  64  65  63]]
[[[108  81  99]
  [109  82 100]
  [116  87 105]
  ...
  [231 189 237]
  [242 190 235]
  [243 191 236]]

 [[107  82 102]
  [109  84 104]
  [112  87 105]
  ...
  [231 189 239]
  [239 189 235]
  [240 190 237]]

 [[107  83 107]
  [110  85 110]
  [110  87 107]
  ...
  [234 189 240]
  [238 189 240]
  [237 187 239]]

 ...

 [[ 82  66  92]
  [ 81  65  90]
  [ 85  63  89]
  ...
  [ 72  46  89]
  [ 71  42  92]
  [ 72  43  94]]

 [[ 81  68  86]
  [ 82  69  87]
  [ 86  67  88]
  ...
  [ 73  48  88]
  [ 72  43  94]
  [ 73  45  95]]

 [[ 79  66  85]
  [ 82  69  87]
  [ 88  70  89]
  ...
  [ 71  50  85]
  [ 67  46  95]
  [ 68  47  96]]]
[[ 89  90  96 ... 208 209 210]
 [ 91  93  95 ... 209 208 210]
 [ 93  95  96 ... 209 210 208]
 ...
 [ 76  74  73 ...  62  60  62]
 [ 75  76  75 ...  63  62  63]
 [ 73  76  78 ... 

In [23]:
cap.release()
cv2.destroyAllWindows()

In [22]:
print (model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 46, 46, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 44, 44, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 22, 22, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 20, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 128)         1

In [38]:
!pip install PyYAML

In [45]:
from tensorflow.keras.models import model_from_yaml

In [46]:
# serialize model to YAML
model_yaml = model.to_yaml()
with open("model.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [48]:
# later...
 
# load YAML and create model
yaml_file = open('model.yaml', 'r')
loaded_model_yaml = yaml_file.read()
yaml_file.close()
loaded_model = model_from_yaml(loaded_model_yaml)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk
